In [ ]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
# Chọn file kaggle.json tải về ở bước 2 để tải lên

!mkdir '/root/.kaggle'
!cp kaggle.json '/root/.kaggle'
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets list
# Chạy lệnh liệt kê các bộ dữ liệu của bạn để đảm bảo bạn đã cài đặt thành công

In [ ]:
!kaggle datasets download -d hasibalmuzdadid/shoe-vs-sandal-vs-boot-dataset-15k-images

 70% 33.0M/47.0M [00:00<00:00, 110MB/s] 
100% 47.0M/47.0M [00:00<00:00, 122MB/s]


In [ ]:
!unzip -q shoe-vs-sandal-vs-boot-dataset-15k-images.zip

In [ ]:
raw_folder = "/content/Shoe vs Sandal vs Boot Dataset/"

In [ ]:
def save_data(raw_folder=raw_folder):
    dest_size=(128,128)
    print("Start Processing")
    pixels = []
    labels = []

    #load tung thu muc con
    for folder in listdir(raw_folder):
        if folder!='.DS_Store':
            print("Folder=", folder)
            # load cac anh
            for file in listdir(raw_folder+folder):
                print("File=",file)
                pixels.append(cv2.resize(cv2.imread(raw_folder+folder+"/"+file), dsize=(dest_size)))
                labels.append(folder)

    pixels=np.array(pixels)
    labels = np.array(labels) #reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    # print(labels)

    file = open('pix.data', 'wb')
    # ket xuat thong tin vao tep do
    pickle.dump((pixels, labels), file)
    # dong file
    file.close()
    return

def load_data():
    file = open('pix.data', 'rb')
    (pixels, labels)= pickle.load(file)
    file.close()
    # print(pixels.shape)
    # print(labels.shape)
    return pixels, labels

In [ ]:
save_data()
X,y=load_data()
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=100)
print(X_train.shape)
print(y_train.shape)

In [ ]:
def get_model():
    model_vgg16_conv=VGG16(weights='imagenet', include_top=False)

    #dong bang cac layer
    for layer in model_vgg16_conv.layers:
        layer.trainable=False

    #tao model
    input = Input(shape=(128,128,3), name='image_input')
    output_vgg16_conv=model_vgg16_conv(input)

    #them cac layer FC va Dropout
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(3, activation='softmax', name='predictions')(x)

    #compile
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return my_model


In [ ]:
vggmodel = get_model()

In [ ]:


filepath = "weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#xay dung dao tao hinh anh de tang cuong du lieu
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
                         rescale=1./255,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val=ImageDataGenerator(rescale=1./255)
vggmodel.fit_generator(aug.flow(X_train, y_train, batch_size= 64),
                                         epochs= 50,
                                         validation_data = aug.flow(X_test, y_test,
                                                                    batch_size=64),
                                callbacks=callbacks_list)


188/188 [==============================] - ETA: 0s - loss: 0.1570 - accuracy: 0.9457
Epoch 50: val_accuracy did not improve from 0.95900
188/188 [==============================] - 65s 348ms/step - loss: 0.1570 - accuracy: 0.9457 - val_loss: 0.1314 - val_accuracy: 0.9537


In [ ]:

vggmodel.save("vggmodel.h5")

In [ ]:
vggmodel.save("vggmodel.h5")

In [ ]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from keras.models import  load_model
import sys


In [ ]:
my_model = get_model()
my_model.load_weights("/content/weights-41-0.96.hdf5")


In [ ]:
image_org = cv2.imread('/content/Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (1).jpg')

image_org = cv2.resize(image_org, dsize=None,fx=0.5,fy=0.5)
# Resize
image = image_org.copy()
image = cv2.resize(image, dsize=(128, 128))
image = image.astype('float')*1./255
# Convert to tensor
image = np.expand_dims(image, axis=0)
class_name = ['boot', 'sandal', 'shoe']
# Predict
predict = my_model.predict(image)
# print("This picture is: ", class_name[np.argmax(predict[0])])
print(np.max(predict[0]))
if (np.max(predict)>=0.8) :
# Show image
    
    print(class_name[np.argmax(predict[0])])

    

1/1 [==============================] - 0s 77ms/step
0.98796785
shoe
